## Dependencias 

In [1]:
import numpy as np 
import pandas as pd
from functools import reduce
import json 
from sqlalchemy.engine import create_engine

pd.set_option('display.max_columns', None)

## Lectura de datos

In [2]:
df = pd.read_csv('data/sme_survey_mx_clean.csv')
df.shape

(885, 22)

In [3]:
df.head(2)

,id,edadEmprendedor,sexoEmprendedor,escolaridadEmprendedor,dependientesEconomicos,estadoCivil,familiaAyuda,antiguedadNegocio,horaApertura,horaCierre,giroNegocio,numEmpleados,ventasPromedioDiarias,deseoNegocio,registroVentas,registroContabilidad,altaSAT,usaCredito,tiempoCreditoProveedores,promocionNegocio,latitud,longitud
0,E00000000,54,H,universidad trunca,3,casado,False,35.0,9,18,"servicios para el hogar (cerrajería, plomería,...",3,3500.0,eficacia,en un cuaderno,en un sistema especial,s,False,no me dan crédito,publicidad exterior|redes sociales (sin pagar)...,19.453446,-99.168899
1,E00000001,56,M,universidad terminado,2,viudo,False,37.0,8,15,comercio minorista de materias primas,3,4500.0,bodega,en excel,en excel,s,False,no me dan crédito,publicidad exterior|redes sociales (sin pagar),19.486575,-99.204668


## Dimensiones 

In [4]:
df.columns

Index(['id', 'edadEmprendedor', 'sexoEmprendedor', 'escolaridadEmprendedor',
       'dependientesEconomicos', 'estadoCivil', 'familiaAyuda',
       'antiguedadNegocio', 'horaApertura', 'horaCierre', 'giroNegocio',
       'numEmpleados', 'ventasPromedioDiarias', 'deseoNegocio',
       'registroVentas', 'registroContabilidad', 'altaSAT', 'usaCredito',
       'tiempoCreditoProveedores', 'promocionNegocio', 'latitud', 'longitud'],
      dtype='object')

In [5]:
dims = ['edadEmprendedor', 'sexoEmprendedor', 'escolaridadEmprendedor',
       'dependientesEconomicos', 'estadoCivil', 'familiaAyuda',
       'antiguedadNegocio', 'horaApertura', 'horaCierre', 'giroNegocio',
       'numEmpleados','registroVentas', 'registroContabilidad', 'altaSAT', 'usaCredito',
       'tiempoCreditoProveedores']

### Edad 

In [6]:
df['edadEmprendedor']
df['dim_edad'] = pd.cut(df['edadEmprendedor'],bins=[-99,30,45,60,99],include_lowest=True).astype(str)
df['dim_edad'].value_counts()

(45.0, 60.0]       336
(30.0, 45.0]       298
(60.0, 99.0]       131
(-99.001, 30.0]    120
Name: dim_edad, dtype: int64

### Sexo

In [7]:
df['dim_sexo'] = df['sexoEmprendedor']

### Escolaridad

In [8]:
mapaEscolaridad = {
        'universidad trunca':'bachillerato', 
        'universidad terminado':'universidad', 
        'secundaria':'básica',
        'bachillerato terminado':'bachillerato', 
        'primaria':'básica', 
        'bachillerato trunco':'básica',
       'posgrado':'universidad'}
df['dim_escolaridad'] = df['escolaridadEmprendedor'].replace(mapaEscolaridad)
df['dim_escolaridad'].value_counts(1)    

básica          0.509605
bachillerato    0.331073
universidad     0.159322
Name: dim_escolaridad, dtype: float64

### Dependientes económicos

In [9]:
df['dim_dependientes'] = df['dependientesEconomicos'].map(lambda x: '5+' if x>=5 else str(x))
df['dim_dependientes'].value_counts(1).sort_index()

0     0.140113
1     0.180791
2     0.277966
3     0.193220
4     0.109605
5+    0.098305
Name: dim_dependientes, dtype: float64

### Estado civil 

In [10]:
df['dim_estado_civil'] = df['estadoCivil']

### Ayuda de la familia

In [11]:
df['dim_familia_ayuda'] = df['familiaAyuda']

### Antigüedad del negocio

In [12]:
df['dim_antiguedad'] = pd.cut(df['antiguedadNegocio'],bins=[-99,2,5,10,99],include_lowest=True).astype(str)
df['dim_antiguedad'].value_counts(1).sort_index()

(-99.001, 2.0]    0.186441
(10.0, 99.0]      0.409040
(2.0, 5.0]        0.177401
(5.0, 10.0]       0.227119
Name: dim_antiguedad, dtype: float64

### Horas de apertura y cierre

In [13]:
df['dim_hora_apertura'] = df['horaApertura'].map(lambda x: '<=7' if x<=7 else '11+' if x>=11 else str(x))
df['dim_hora_apertura'].value_counts(1).sort_index()

10     0.161582
11+    0.215819
8      0.228249
9      0.236158
<=7    0.158192
Name: dim_hora_apertura, dtype: float64

In [14]:
df['dim_hora_cierre'] = df['horaCierre'].map(lambda x: '<=17' if x<=17 else '22+' if x>=22 else str(x))
df['dim_hora_cierre'].value_counts(1).sort_index()

18      0.168362
19      0.138983
20      0.162712
21      0.105085
22+     0.153672
<=17    0.271186
Name: dim_hora_cierre, dtype: float64

### Giro del negocio

In [15]:
df['dim_giro'] = df['giroNegocio'].replace(
  {
    'servicios para el hogar (cerrajería, plomería, jardinería, etc)': 'Servicios para el hogar',
    'comercio minorista de materias primas': 'Comercio minorista',
    'servicios  personales(estéticas, spa, uñas, barberías, etc)': 'Servicios personales',
    'comercio minorista de productos terminados': 'Comercio minorista',
    'fonda o restaurante/alimentos': 'Alimentos y bebidas',
    'reparación de celulares.': 'Servicios para el hogar',
    'estampados y publicidad.': 'Servicios comerciales',
    'consultorios médicos, dentales, veterinarios': 'Servicios de salud',
    'farmacia ': 'Salud y belleza',
    'planchaduría ': 'Servicios personales',
    'lonas y reparación ': 'Servicios comerciales',
    'planchaduría': 'Servicios personales',
    'librería ': 'Comercio minorista',
    'papelería ': 'Comercio minorista',
    'cafe internet': 'Servicios de telecomunicaciones',
    'frenos': 'Servicios para el hogar',
    'escuela de manejo': 'Educación',
    'vulcanizadora': 'Servicios para el hogar',
    'negocio mercantil con giro comercial en venta de tortillas': 'Comercio minorista',
    'pasteleria': 'Alimentos y bebidas',
    'costurero': 'Servicios personales',
    'cafe internet ': 'Servicios de telecomunicaciones',
    'mecánico ': 'Servicios para el hogar',
    'sastrería ': 'Servicios personales',
    'publicidad ': 'Servicios comerciales',
    'café internet ': 'Servicios de telecomunicaciones',
    'purificadora de agua.': 'Servicios para el hogar',
    'brujería': 'Otros',
    'reparaciones electrónicas.': 'Servicios para el hogar',
    'taller de motocicletas': 'Servicios para el hogar',
    'jarceria, productos de limpieza': 'Comercio minorista',
    'café internet': 'Servicios de telecomunicaciones',
    'manufacturera ': 'Comercio minorista',
    'internet': 'Servicios de telecomunicaciones',
    'productos de limpieza ': 'Comercio minorista',
    'artículos de limpieza ': 'Comercio minorista',
    'semillas ': 'Comercio minorista',
    'calzado ': 'Comercio minorista',
    'sandalias': 'Comercio minorista',
    'tlapaleria  ': 'Comercio minorista',
    'accesorios para mascotas ': 'Comercio minorista',
    'rea pa uno de zapatos ': 'Servicios personales',
    'repostería ': 'Alimentos y bebidas',
    'accesorios telefónicos ': 'Comercio minorista',
    'tienda': 'Comercio minorista',
    'regalos': 'Comercio minorista',
    'pintura': 'Comercio minorista',
    'verdulería ': 'Comercio minorista',
    'productos decorativos ': 'Comercio minorista',
    'cremeria ': 'Alimentos y bebidas',
    'banquetes': 'Alimentos y bebidas',
    'suspensiones y ruedas de méxico': 'Servicios para el hogar',
    'accesorios para mascota': 'Comercio minorista',
    'farmacia': 'Salud y belleza',
    'cibercafé': 'Servicios de telecomunicaciones',
    'pictórica': 'Otros',
    'jarcería': 'Comercio minorista',
    'telecomunicaciones ': 'Servicios de telecomunicaciones',
    'zapatería ': 'Comercio minorista',
    'bazar': 'Comercio minorista',
    'abarrotes ': 'Comercio minorista',
    'recaudería': 'Comercio minorista',
    'recaudería ': 'Comercio minorista',
    'reparadora de calzado ': 'Servicios para el hogar',
    'internet ': 'Servicios de telecomunicaciones',
    'estética ': 'Servicios personales',
    'carnicería ': 'Alimentos y bebidas',
    'pollería ': 'Alimentos y bebidas',
    'reparadora de electrodomésticos ': 'Servicios para el hogar',
    'pastelería ': 'Alimentos y bebidas',
    'herrajes y tlapalería': 'Comercio minorista',
    'tortillería ': 'Comercio minorista',
    'rosticeria ': 'Alimentos y bebidas',
    'servicios de reparación': 'Servicios para el hogar',
    'automotriz': 'Servicios para el hogar',
    'tintoreria y limpieza': 'Servicios personales',
    'academia de danza': 'Educación',
    'tlapaleria ': 'Comercio minorista',
    'salud y belleza': 'Salud y belleza',
    'frutas y verduras ': 'Comercio minorista',
    'ropa': 'Comercio minorista',
    'de temporada': 'Comercio minorista',
    'veterinaria': 'Servicios de salud',
    'despacho jurídico ': 'Servicios profesionales',
    'despacho contable': 'Servicios profesionales',
    'tortillería': 'Comercio minorista',
    'panadería': 'Alimentos y bebidas',
    'reparaciones celulares': 'Servicios para el hogar',
    'estetica': 'Servicios personales'
}
)
df['dim_giro'].value_counts(1).sort_index()

Alimentos y bebidas                0.179661
Comercio minorista                 0.575141
Educación                          0.002260
Otros                              0.002260
Salud y belleza                    0.010169
Servicios comerciales              0.003390
Servicios de salud                 0.048588
Servicios de telecomunicaciones    0.010169
Servicios para el hogar            0.068927
Servicios personales               0.097175
Servicios profesionales            0.002260
Name: dim_giro, dtype: float64

### Empleados

In [16]:
df['dim_empleados'] = df['numEmpleados'].map(lambda x: '3+' if x>=3 else str(x))
df['dim_empleados'].value_counts(1).sort_index()

0     0.275706
1     0.258757
2     0.245198
3+    0.220339
Name: dim_empleados, dtype: float64

### Registro de ventas y contabilidad 

In [17]:
for x in ['Ventas','Contabilidad']:
    df[f'dim_{x.lower()}'] = df[f'registro{x}']
    display(df[f'dim_{x.lower()}'].value_counts(1).sort_index())

en excel                  0.110734
en un cuaderno            0.517514
en un sistema especial    0.091525
no registro               0.280226
Name: dim_ventas, dtype: float64

en excel                  0.160452
en un cuaderno            0.436158
en un sistema especial    0.131073
no registro               0.272316
Name: dim_contabilidad, dtype: float64

### SAT

In [18]:
df['dim_sat'] = df['altaSAT']

### Crédito

In [19]:
df['dim_credito'] = df['usaCredito']

### Tiempo de crédito

In [20]:
df['dim_credito_proveedor'] = df['tiempoCreditoProveedores']

### Promoción del negocio

In [21]:
medios = reduce(lambda x,y: x.union(y),df['promocionNegocio'].map(lambda x:x.split('|')).map(set))

In [43]:
aux = df[['id','promocionNegocio']].copy()
aux[list(range(4))] = aux['promocionNegocio'].map(lambda x: x.split('|')).apply(pd.Series)
aux = aux.melt(id_vars=['id'],value_vars=list(range(4)),value_name='dim_medio').dropna().drop(columns='variable').drop_duplicates().sort_values(by='id').reset_index(drop=True)
cat_medios = aux[['dim_medio']].drop_duplicates().reset_index(drop=True)
cat_medios['id_medio'] = cat_medios.index+1   
aux = aux.merge(cat_medios,on='dim_medio',how='inner').drop(columns='dim_medio').rename(columns={'id':'id_emprendedor'})
aux['id'] = aux.index+1
aux

,id_emprendedor,id_medio,id
0,E00000000,1,1
1,E00000001,1,2
2,E00000002,1,3
3,E00000003,1,4
4,E00000006,1,5
...,...,...,...
1152,E00000880,4,1153
1153,E00000881,4,1154
1154,E00000882,4,1155
1155,E00000883,4,1156


## Hechos (métricas )

In [23]:
hechos = ['edadEmprendedor','ventasPromedioDiarias']

## Armado de cubos

In [36]:
creds = json.load(open('creds.json','r'))
cnx = create_engine(f"mysql+mysqlconnector://{creds['u']}:{creds['p']}@{creds['h']}:3306/{creds['d']}").connect()

In [37]:
central = df.copy()
central.rename(columns={'id':'id_emprendedor'},inplace=True)
for c in central.filter(like='dim_').columns:
    cat = central[[c]].drop_duplicates().reset_index(drop=True)
    cat.insert(0,'id',cat.index+1)
    central = central.merge(cat,on=c,how='inner').drop(c,axis=1).rename(columns={'id':f'id_{"_".join(c.split("_")[1:])}'})
    cat.to_sql(f'cat_{"_".join(c.split("_")[1:])}',cnx,if_exists='replace',index=False)

In [25]:
central = central.filter(like='id_')

In [26]:
central.head()

,id_emprendedor,id_edad,id_sexo,id_escolaridad,id_dependientes,id_estado_civil,id_familia_ayuda,id_antiguedad,id_hora_apertura,id_hora_cierre,id_giro,id_empleados,id_ventas,id_contabilidad,id_sat,id_credito,id_credito_proveedor
0,E00000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,E00000811,1,2,2,4,1,2,3,5,1,2,1,1,1,1,1,1
2,E00000212,1,1,2,3,2,2,4,4,4,2,1,1,1,1,1,1
3,E00000272,2,2,2,6,1,2,4,2,6,2,1,1,1,1,1,1
4,E00000616,1,1,2,2,1,2,1,4,2,4,2,1,1,1,1,1


In [27]:
tbl_hechos = df[['id']+hechos].rename(columns={'id':'id_emprendedor'})

In [28]:
tbl_hechos.head()

,id_emprendedor,edadEmprendedor,ventasPromedioDiarias
0,E00000000,54,3500.0
1,E00000001,56,4500.0
2,E00000002,46,26250.0
3,E00000003,49,1000.0
4,E00000004,47,12000.0


In [29]:
dims = list(central.columns)[1:]

In [30]:
cubo = central.merge(tbl_hechos,on='id_emprendedor').groupby(dims).agg(**{'count_pymes':('id_emprendedor','count'),
                                                                   'sum_edad':('edadEmprendedor','sum'),
                                                                   'sum_ventas':('ventasPromedioDiarias','sum'),
                                                                   }
                                                                   ).reset_index()

In [38]:
cubo.to_sql('cubo',cnx,if_exists='replace',index=False)

884

In [40]:
aux

,id_emprendedor,id_medio,id
0,E00000000,1,1
1,E00000001,1,2
2,E00000002,1,3
3,E00000003,1,4
4,E00000006,1,5
...,...,...,...
1152,E00000880,4,1153
1153,E00000881,4,1154
1154,E00000882,4,1155
1155,E00000883,4,1156


In [44]:
aux.to_sql('tbl_medios',cnx,if_exists='replace',index=False)

1157

In [47]:
cat_medios.to_sql('cat_medios',cnx,if_exists='replace',index=False)

4

In [46]:
cat_medios

,dim_medio,id_medio
0,publicidad exterior,1
1,volantes,2
2,redes sociales (sin pagar),3
3,otro,4
